In [49]:
from typing import Tuple, List
from AcasXuNet import AcasXu
from ReluPatterns import Patterns
import torch
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
PATH = '/home/allen/Desktop/VerifyNNE-master/Marabou/resources/nnet/mnist/mnist4x256.nnet'
network = AcasXu(PATH)
#print(network.means)
#print("min", network.mins)
#print("max", network.maxs)

dataset: Tuple[Tensor] = torch.load('/home/allen/Desktop/VerifyNNE-master/training.pt')
new_data1 = torch.reshape(dataset[0], (60000,784))


#new_data1 = new_data1.type(torch.DoubleTensor)

new_data_un = new_data1.type(torch.FloatTensor)

# new_data1 = new_data_un
new_data1 = new_data_un[:,:]/255.

#print(new_data1[0])

dataset_train = TensorDataset(new_data1, dataset[1])
print(new_data1.shape, dataset[1].shape)
#print(dataset_train)


# print(new_data1[6,:])

loader = DataLoader(dataset_train, batch_size=2000, shuffle=True)












torch.Size([60000, 784]) torch.Size([60000])


In [50]:



# all_patterns: Patterns = Patterns(model = network,
#                         dataloader = loader,
#                         labels = range(10),
#                         layers = ['layer0', 'layer1', 'layer2', 'layer3', 'layer4','layer5','layer6', 'layer7', 'layer8', 'layer9'])

    
    
all_patterns: Patterns = Patterns(model = network,
                        dataloader = loader,
                        labels = range(10),
                        layers = ['layer0', 'layer1'])
    
    

all_patterns: Patterns = Patterns(model = network,
                        dataloader = loader,
                        labels = range(10),
                        layers = ['layer0', 'layer1', 'layer2', 'layer3'])

print(all_patterns)


all_preds: List[int] = []
true_labels: List[int] = []
network.register_log()
for inputs, outputs in loader:
    raw_pred = network(inputs)
    pred: List[int] = torch.argmax(raw_pred, dim = 1).squeeze().tolist()
    all_preds.extend(pred)
    true_labels.extend(outputs.squeeze().tolist())

print(classification_report(true_labels, all_preds))
print(accuracy_score(true_labels, all_preds))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5923
           1       1.00      1.00      1.00      6742
           2       0.99      0.99      0.99      5958
           3       1.00      0.98      0.99      6131
           4       0.99      0.99      0.99      5842
           5       0.98      0.99      0.99      5421
           6       0.99      1.00      0.99      5918
           7       0.99      1.00      0.99      6265
           8       0.99      0.99      0.99      5851
           9       0.99      0.98      0.99      5949

    accuracy                           0.99     60000
   macro avg       0.99      0.99      0.99     60000
weighted avg       0.99      0.99      0.99     60000

0.99085


In [52]:
import json


EPSILON = 0.005
alpha_patterns = {}
for label in all_patterns.label2patterns:
    patterns = all_patterns.label2patterns[label]
    print(patterns.shape)


    relu_sum = np.sum(patterns, axis = 0).squeeze()

    stable_idx = np.concatenate([np.where(relu_sum<=EPSILON*patterns.shape[0]), 
                                    np.where(relu_sum>=(1-EPSILON)*patterns.shape[0])],
                                axis = 1
                                ).squeeze()
    print(stable_idx)
    print(relu_sum[stable_idx])
    alpha_patterns[label] = {"stable_idx": stable_idx.tolist(), "val": relu_sum[stable_idx].tolist()}

with open("MNIST_NAP/mnist_relu_patterns_" + str(EPSILON) + ".json", "w") as f:
    json.dump(alpha_patterns, f, indent=2)

(5923, 1024)
[   4   17   18   19   22   23   28   31   32   38   41   45   47   48
   51   55   63   65   72   73   79   82   98  100  106  110  115  116
  125  141  142  143  145  149  150  152  164  202  206  209  212  224
  225  227  236  241  243  244  252  257  271  272  280  282  284  289
  303  307  318  325  333  335  356  361  363  365  375  380  381  407
  414  429  431  448  453  454  483  492  512  513  514  515  516  517
  518  519  520  521  522  524  525  526  527  528  530  531  532  533
  534  535  536  537  538  539  540  541  542  544  545  546  548  549
  550  551  552  553  554  556  557  558  559  560  561  562  563  564
  565  566  567  568  569  570  571  572  574  575  576  577  578  579
  580  581  582  583  584  585  587  588  589  590  591  592  593  594
  595  596  597  598  599  600  601  602  603  604  605  606  607  608
  609  610  611  612  613  614  615  616  617  618  620  621  622  623
  624  625  626  627  628  629  630  631  632  633  634  635  63